In [ ]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np

In [ ]:
def crea_dataframe(data): #data -> ruta del archivo
    #creacion dataframe, elimina columnas Lat y Long 
    df = pd.read_csv(data,index_col="Country/Region",parse_dates=True).drop(["Lat","Long"],axis=1)

    #reduce la columna province/state con un groupby manteniendo los totales por pais
    df = df.groupby(level=0).sum(numeric_only=True)

    #parsear fechas
    fechas = []
    for i in df.columns:
        fechas.append(pd.to_datetime(i)) #añade a una lista la transformacion a timestamp de cada fecha
    df.columns = fechas #asigna a columnas la lista de fechas parseadas a timestamp

    return df
        

In [ ]:
df_confirmados = crea_dataframe("data/covid19_confirmados.csv") #creacion del df
df_population = pd.read_csv("data/poblacion_mundial2020.csv") #dataframe de population
df_population.rename(columns={"Country (or dependency)":"Country/Region"}, inplace=True) #Cambio del nombre de el indice de la columna
df_population.set_index("Country/Region",inplace=True) #asignacion de "nuevo" nombre

In [ ]:
def eliminar_error(df,df_population):
    valores_error = []
    for pais in df.index:
        if pais not in df_population.index:
            valores_error.append(pais)
            df.drop(labels=pais, axis=0, inplace=True)

In [ ]:
eliminar_error(df_confirmados,df_population)
df_confirmados = pd.merge(df_confirmados,df_population, on="Country/Region")

In [ ]:
#creacion de siguientes dataframe y merge con population
df_recuperados = crea_dataframe("data/covid19_recuperados.csv") #creacion del df
eliminar_error(df_recuperados,df_population)
df_recuperados = pd.merge(df_recuperados,df_population, on="Country/Region") #merge

df_muertes = crea_dataframe("data/covid19_muertes.csv") #creacion del df
eliminar_error(df_muertes,df_population)
df_muertes = pd.merge(df_muertes,df_population, on="Country/Region") #merge

#

In [ ]:
#Continentes y codigo ISO2 archivo generado con chatgpt
continentes = pd.read_csv("data/pais_cod_continente.csv",index_col="Country/Region")

df_confirmados = pd.merge(df_confirmados,continentes,on="Country/Region")
df_recuperados = pd.merge(df_recuperados,continentes,on="Country/Region")
df_muertes = pd.merge(df_muertes,continentes,on="Country/Region")

In [ ]:
#dataframe de paises OCDE -> datos obtenidos con chatgpt
df_OCDE_confirmados = pd.read_csv("data/paises_OCDE.csv",index_col="Country/Region")
df_OCDE_confirmados = pd.merge(df_confirmados,df_OCDE_confirmados,on="Country/Region")

df_OCDE_recuperados= pd.read_csv("data/paises_OCDE.csv",index_col="Country/Region")
df_OCDE_recuperados = pd.merge(df_recuperados,df_OCDE_recuperados,on="Country/Region")

df_OCDE_muertes = pd.read_csv("data/paises_OCDE.csv",index_col="Country/Region")
df_OCDE_muertes = pd.merge(df_muertes,df_OCDE_muertes,on="Country/Region")

In [ ]:
dataframes_OCDE = {0: df_OCDE_confirmados,
                   1: df_OCDE_recuperados,
                   2: df_OCDE_muertes} #ayuda a acceder al df de forma mas facil
plt.ioff()
####################################### Filtros #############################################

def filtro1(df): #casos nuevos o casos acumulados
    if(selector_casos.value == "Casos nuevos"):
        return df.diff(axis=1).clip(lower=0) # diff() calcula la diferencia entre filas (o columnas) adyacentes axis = 0 -> filas axis = 1 columnas
        #clip() limita los valores de una columna a un rango específico.  
        #En este caso, se puede establecer el rango mínimo en cero para eliminar los valores negativos.
        #los valores negativos son dados debido a falla en los datos si fecha1 x-1 y fecha2 x al hacer la diferencia para obtener nuevos
        #casos fecha2 - fecha1 el resultado da negativo
    elif(selector_casos.value == "Casos acumulados"):
        return df
    else:
        return df

def filtro2(df):
    if(selector_valores.value == "Valores absolutos"):
        df.loc[pais1.value]  = df.loc[pais1.value][:810]*100000/df_OCDE_confirmados.loc[pais1.value]["Population (2020)"]
        df.loc[pais2.value] = df.loc[pais2.value][:810]*100000/df_OCDE_confirmados.loc[pais2.value]["Population (2020)"]

        return df
    elif(selector_valores.value == "Valores relativos"):
        return df
    else:
        return df
    
def filtro3(df):
    if(selector_fecha.value == "Valores semanales"):
        fechas = pd.PeriodIndex(df.columns, freq="W-MON", copy = bool).to_timestamp() #Genera un arreglo de tipo datetime con las fechas del dataframe(columnas)
        #con frecuencia semanal con el dia lunes como dia inicial
        df.columns = fechas
        df.groupby(df.columns , axis=1).sum() #agrupar en una suma los valores de las fechas con el mismo nombre de columna(axis=1 es columnas)
        
        return df
    elif(selector_fecha.value == "Valores diarios"):
        return df
    else:
        return df

def crear_grafico(num): #crea el grafico dentro de cada hijo de la tabla
    if(pais1.value == None or pais2.value == None):
        display("selecciona paises")
    elif(pais1.value == pais2.value):
        display("selecciona paises distintos")
    else:
        with tabla.children[num]:
            if(selector_casos.value != None or selector_valores.value != None or selector_fecha.value != None):
                df_OCDE_c = dataframes_OCDE[num].copy()

                #Aplicacion de filtros
                df_OCDE_c = filtro1(df_OCDE_c.iloc[:,: 810])

                df_OCDE_c = filtro2(df_OCDE_c.iloc[:,: 810])

                df_OCDE_c = filtro3(df_OCDE_c.iloc[:,: 810])
                ####################################################

                fig, ax = plt.subplots(figsize=(14, 8), tight_layout=True, facecolor='#EEF')
                
                line, = ax.plot(df_OCDE_c.loc[pais1.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")],label=pais1.value)
                line2, = ax.plot(df_OCDE_c.loc[pais2.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")], label=pais2.value)
                plt.legend()
                ax.set_title(pais1.value+" vs "+pais2.value)
                plt.show()
            else:
                fig, ax = plt.subplots(figsize=(14, 8), tight_layout=True, facecolor='#EEF')
                
                line, = ax.plot(dataframes_OCDE[num].loc[pais1.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")],label=pais1.value)
                line2, = ax.plot(dataframes_OCDE[num].loc[pais2.value, pd.to_datetime("2020-01-22"): pd.to_datetime("2022-04-10")], label=pais2.value)
                plt.legend()
                ax.set_title(pais1.value+" vs "+pais2.value)
                plt.show()

def boton_presionado(dato): #elimina el grafico actual y crea otro grafico con los nuevos valores
    for i in range(0,3):
        with tabla.children[i]:
            tabla.children[i].clear_output()
            crear_grafico(i)

def r_filtros(data):
    selector_casos.value = None
    selector_valores.value = None
    selector_fecha.value = None

########################################## Creacion de botones y tabla ##########################################

pais1 = widgets.Dropdown(value=None, options=df_OCDE_confirmados.index, description='Pais 1')
pais2 = widgets.Dropdown(value=None, options=df_OCDE_confirmados.index, description='Pais 2')

confirmar = widgets.Button(description='Confirmar')
reiniciar_filtros = widgets.Button(description='Reiniciar filtros')

selector_casos = widgets.RadioButtons(
    options=['Casos acumulados', 'Casos nuevos'],
    value=None,
    disabled=False
)

selector_valores = widgets.RadioButtons(
    options=['Valores absolutos', 'Valores relativos'],
    value=None,
    disabled=False
)

selector_fecha = widgets.RadioButtons(
    options=['Valores diarios', 'Valores semanales'],
    value=None,
    disabled=False
)

datos_tabla = ["Tasa incidencias", "Tasa recuperados","Tasa mortalidad"]
tabla = widgets.Tab(children=[widgets.Output() for i in range(len(datos_tabla))])
for i in range(len(datos_tabla)):
    tabla.set_title(i,datos_tabla[i])

output = widgets.Output()

fila1 = widgets.HBox([pais1,pais2,output])
fila_filtro = widgets.VBox([confirmar,reiniciar_filtros])
fila2 = widgets.HBox([selector_casos,selector_valores,selector_fecha,fila_filtro])

display(widgets.VBox([fila1, fila2,output]),tabla)

#################################### Acciones con cada boton ####################################

confirmar.on_click(boton_presionado) #asigna los filtros seleccionados a cada copia del dataframe
reiniciar_filtros.on_click(r_filtros) #asigna none a los valores de los filtros
